In [233]:
import os
import pandas as pd
import string
import unicodedata
import re

folder_path = 'Forms/AV1'
files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

In [234]:
def preprocess_column(list_):

    p = list()

    for el in list_:

        v = str(el).translate(str.maketrans('', '', string.punctuation)).lower()
        v = unicodedata.normalize('NFD', v)
        v = ''.join(c for c in v if unicodedata.category(c) != 'Mn')

        v = v.strip()
        v = re.sub(r'\s+', ' ', v)


        p.append(v)

    return p

In [235]:
list_df = list()
d = {}

for file in files:

    df = pd.read_excel(f"{folder_path}/{file}")
    df.columns = preprocess_column(list(df.columns))
    cpf_column = [x for x in list(df.columns) if "cpf" in x]
    assert len(cpf_column) == 1
    df["cpf"] = preprocess_column(list(df[cpf_column[0]]))

    pont_column = [x for x in list(df.columns) if "pontuacao" in x]
    assert len(pont_column) == 1
    df[f"{file}_pontuacao"] = preprocess_column(list(df[pont_column[0]]))

    d_ = df.set_index("cpf")[f"{file}_pontuacao"].to_dict()

    d[file] = d_

In [239]:
for key in list(d.keys()):

    df_d = pd.DataFrame.from_dict(d[key])

    print(len(d[key]))

85
94
83
89
82
87
89


In [241]:
d[key]

{'10729965457': '9',
 '71143562429': '10',
 '11630750433': '5',
 '12422429459': '10',
 '8885476414': '10',
 '9676480339': '10',
 '13824638479': '10',
 '6256907450': '8',
 '13900373477': '10',
 '6537370337': '10',
 '8276023401': '9',
 '12360927400': '10',
 '3490430433': '9',
 '5414697540': '6',
 '70475844475': '9',
 '61436942306': '10',
 '12246436400': '10',
 '10740290541': '10',
 '7640174465': '8',
 '11638329443': '10',
 '6860991330': '10',
 '70513644482': '10',
 '11229057420': '10',
 '3091564240': '10',
 '70922207429': '10',
 '8766383455': '10',
 '6601727393': '10',
 '14657853457': '10',
 '12476409499': '8',
 '8972578479': '10',
 '13255067456': '9',
 '7216257383': '10',
 '5224970555': '10',
 '7214858398': '10',
 '7273607363': '10',
 '7344504359': '9',
 '12950633447': '10',
 '7489111463': '10',
 '14741643409': '10',
 '11143715489': '10',
 '8523294406': '10',
 '13707367460': '9',
 '44681507857': '10',
 '8251418429': '10',
 '70259324426': '10',
 '14182225422': '9',
 '8930254373': '10',
 